CSVs retrieved from: https://data.ontario.ca/dataset/long-term-care-home-covid-19-data

In [14]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-14-284ed33d9215>:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Import

In [15]:
# active = pd.read_csv("https://data.ontario.ca/datastore/dump/4b64488a-0523-4ebb-811a-fac2f07e6d59?bom=True")
active = pd.read_csv("../../data/ON/activeltcoutbreak_oct8.csv")
active.info()
active.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10444 entries, 0 to 10443
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ï»¿Report_Data_Extracted   10444 non-null  object 
 1   LTC_Home                   10444 non-null  object 
 2   LTC_City                   0 non-null      float64
 3   Beds                       10443 non-null  float64
 4   Total_LTC_Resident_Cases   10444 non-null  object 
 5   Total_LTC_Resident_Deaths  10444 non-null  object 
 6   Total_LTC_HCW_Cases        10443 non-null  object 
dtypes: float64(2), object(5)
memory usage: 571.3+ KB


,ï»¿Report_Data_Extracted,LTC_Home,LTC_City,Beds,Total_LTC_Resident_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Cases
0,2020-04-24,"Albright Gardens Homes, Incorporated",NaN,231.0,<5,0,<5
1,2020-04-24,Almonte Country Haven,NaN,82.0,44.0,25,17.0
2,2020-04-24,Altamont Care Community,NaN,159.0,125.0,32,45.0
3,2020-04-24,Anson Place Care Centre,NaN,61.0,28.0,23,33.0
4,2020-04-24,Bay Ridges,NaN,124.0,0.0,0,<5


In [16]:
# inactive = pd.read_csv("https://data.ontario.ca/dataset/42df36df-04a0-43a9-8ad4-fac5e0e22244/resource/0cf2f01e-d4e1-48ed-8027-2133d059ec8b/download/resolvedltc.csv")
resolved = pd.read_csv("../../data/ON/resolvedltc_oct8.csv")
resolved.info()
resolved.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40629 entries, 0 to 40628
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ï»¿Report_Data_Extracted   40629 non-null  object
 1   LTC_Home                   40629 non-null  object
 2   City                       40629 non-null  object
 3   Beds                       40628 non-null  object
 4   Total_LTC_Resident_Deaths  40629 non-null  object
dtypes: object(5)
memory usage: 1.5+ MB


,ï»¿Report_Data_Extracted,LTC_Home,City,Beds,Total_LTC_Resident_Deaths
0,2020-04-24,Allendale,Milton,200.0,0
1,2020-04-24,Chartwell Westmount Long Term Care Residence,Kitchener,160.0,0
2,2020-04-24,Chelsey Park,London,247.0,0
3,2020-04-24,Chester Village,Toronto,203.0,0
4,2020-04-24,Clarion Nursing Home,Stoney Creek,100.0,0


In [17]:
active = active.rename(columns = {'ï»¿Report_Data_Extracted':'Report_Data_Extracted'})
resolved = resolved.rename(columns = {'ï»¿Report_Data_Extracted':'Report_Data_Extracted'})

Clean

In [18]:
# convert to datetime
active.Report_Data_Extracted= pd.to_datetime(active.Report_Data_Extracted)
resolved.Report_Data_Extracted= pd.to_datetime(resolved.Report_Data_Extracted)

In [8]:
# Inspect data
# The number of deaths went from 30 to 29 on June 16?
# active.groupby("LTC_Home").count()
# active.loc[active["LTC_Home"] == "Afton Park Place Long Term Care Community"]
active.loc[active["LTC_Home"] == "Almonte Country Haven"]

,Report_Data_Extracted,LTC_Home,LTC_City,Beds,Total_LTC_Resident_Cases,Total_LTC_Resident_Deaths,Total_LTC_HCW_Cases
1,2020-04-24,Almonte Country Haven,NaN,82.0,44.0,25,17.0
146,2020-04-25,Almonte Country Haven,NaN,82.0,44.0,25,17.0
293,2020-04-26,Almonte Country Haven,NaN,82.0,44.0,25,17.0
442,2020-04-27,Almonte Country Haven,NaN,82.0,44.0,25,17.0
596,2020-04-28,Almonte Country Haven,NaN,82.0,44.0,25,17.0
755,2020-04-29,Almonte Country Haven,NaN,82.0,44.0,25,19.0
918,2020-04-30,Almonte Country Haven,NaN,82.0,44.0,25,19.0
1084,2020-05-01,Almonte Country Haven,NaN,82.0,36.0,25,23.0
1251,2020-05-02,Almonte Country Haven,NaN,82.0,36.0,25,23.0
1421,2020-05-03,Almonte Country Haven,NaN,82.0,45.0,24,18.0


**NOTE:** Inactive outbreaks were reported daily since status change from `active` to `inactive`, leading to multicate entries. For simplicity, the earliest date was kept under the assumption that this date is when the outbreak was declared to be over. 

The same reporting was applied to active outbreaks. Again, the first date was assumed to be the declaration of the start of the outbreak.

**NL**

- Keep rows where the date is July 31

In [19]:
active731 = active.loc[active['Report_Data_Extracted'] == '2020-07-31']
resolved731 = resolved.loc[resolved['Report_Data_Extracted'] == '2020-07-31']

In [20]:
active731.info()
resolved731.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19 entries, 8914 to 8932
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Report_Data_Extracted      19 non-null     datetime64[ns]
 1   LTC_Home                   19 non-null     object        
 2   LTC_City                   0 non-null      float64       
 3   Beds                       19 non-null     float64       
 4   Total_LTC_Resident_Cases   19 non-null     object        
 5   Total_LTC_Resident_Deaths  19 non-null     object        
 6   Total_LTC_HCW_Cases        19 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 1.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 19223 to 19531
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Report_Data_Ex

In [21]:
def returnNotMatches(a, b):
    return [[x for x in a if x not in b], [x for x in b if x not in a]]

active731['LTC_Home'].isin(resolved731['LTC_Home']).value_counts()

False    19
Name: LTC_Home, dtype: int64

In [22]:
outbreaks = pd.concat([active731, resolved731]) 
outbreaks = outbreaks[['LTC_Home', 'Beds']] 

outbreaks.info()
outbreaks.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 8914 to 19531
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   LTC_Home  328 non-null    object
 1   Beds      328 non-null    object
dtypes: object(2)
memory usage: 7.7+ KB


,LTC_Home,Beds
8914,Carleton Lodge,161
8915,Cawthra Gardens,192
8916,Columbia Forest,156
8917,Ã‰lisabeth-BruyÃ¨re Residence,71
8918,Extendicare Falconbridge,234


In [31]:
# Export the outbreak data
outbreaks.to_csv('../merge_LTC_database/ltc_outbreaks0731.csv')

In [13]:
# keep_indices = inactive.LTC_Home.drop_duplicates(keep='first').index.to_list()
# inactiveFiltered = inactive.iloc[keep_indices]

# keep_indices2 = active.LTC_Home.drop_duplicates(keep='first').index.to_list()
# activeFiltered = active.iloc[keep_indices2]

Merge - dropping duplicates where LTCs are in both active and inactive lists, keeping the active home.

First, check that no homes reported more than one outbreak.

**NL**
- Given that we had dropped duplicate `LTC_Home` above from the `active` and `inactive` dfs, would not expect to capture info on homes that reported more than one outbreak

In [23]:
# # Inspect
# # There are 342 homes on the active list and 346 homes on the inactive list
# # Suggests there are homes where outbreaks recorded as resolved were never recorded as having an active outbreak
# inactiveFiltered.info()
# inactiveFiltered.head(20)
# inactiveFiltered.nunique()
# activeFiltered.info()
# activeFiltered.head(20)
# activeFiltered.nunique()

In [24]:
# outbreaks = pd.concat([inactiveFiltered, activeFiltered])
# outbreaks.info()
# outbreaks.head()

In [25]:
# grouped = outbreaks.groupby(["LTC_Home","outbreak_status"]).count()
# grouped.info()
# grouped.head()

In [26]:
# grouped.loc[grouped["Report_Data_Extracted"] > 1]

In [27]:
# # For homes that appear in both active and inactive, keep the most recent classification
# # For inactive homes you have the first date when an outbreak was classified as resolved
# # For acitve homes you have the first date when an outbreak was declared
# outbreaks.drop_duplicates(subset='LTC_Home', keep="last", inplace = True)
# outbreaks.info()

Filter for those reported before August 1, 2020:

**NL**
- If we filter like this do we fail to capture homes that reported on outbreak before Aug 1, but then where the outbreak was reported as being resolved after Aug 1

In [28]:
# outbreaksAug1 = outbreaks.loc[outbreaks['Report_Data_Extracted'] < "2020-08-01"]

Import LTC data and merged with outbreaks:

In [ ]:
# ltc = pd.read_csv("../merge_LTC_database/webscrape_LTC_general_database.csv")

In [ ]:
# outbreaksAug1["name"] = outbreaksAug1.LTC_Home.str.upper()
# complete = pd.merge(ltc, outbreaksAug1, on = "name")

In [ ]:
# print("Number of homes in outbreak DF missing after merge: ", len(set(outbreaksAug1.name.unique())-set(complete.name.unique())))

Fix name issues and re-merge

In [ ]:
# list(set(outbreaksAug1.name.unique())-set(complete.name.unique()))

In [ ]:
# ltc.name.replace({'RESIDENCE SAINT- LOUIS':'RESIDENCE SAINT-LOUIS',
#                  'FINLANDIA HOIVAKOTI NURSING HOME LIMITED': 'FINLANDIA HOIVAKOTI NURSING HOME',
#                   'PINECREST NURSING HOME - BOBCAYGEON': 'PINECREST NURSING HOME (BOBCAYGEON)',
#                  'ST PATRICK\'S HOME': "ST. PATRICK'S HOME",
#                   'VISION \'74 INC.':'VISION NURSING HOME',
#                   'MACASSA LODGE':'MACASSA  LODGE',
#                   'HEARTWOOD (FKA VERSA-CARE CORNWALL)':'HEARTWOOD'
#                  },
#                 inplace = True)

In [ ]:
# complete = pd.merge(ltc, outbreaksAug1, on = "name")
# print("Number of homes in outbreak DF missing after merge: ", len(set(outbreaksAug1.name.unique())-set(complete.name.unique())))
# list(set(outbreaksAug1.name.unique())-set(complete.name.unique()))

**Clean & Export:**

In [ ]:
# complete.drop(columns = ['_id','LTC_City','additional_info','LTC_Home',
#                         'management', 'city', 'City', 'index'], inplace = True)
# complete.set_index('name', inplace = True)

In [ ]:
# complete.to_csv('../merge_LTC_database/LTC_general_DB_aug1.csv')